In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,9
plt.rcParams['font.size'] = 22

In [84]:
# https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions
import numpy as np
from scipy.ndimage import zoom


def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [85]:
rot_tgts = pd.read_csv('sphere/2percent_removed/rotation_targets.csv')

In [86]:
rot_tgts.keys()

Index(['# ids', 'rot_voxel', 'cube_x', 'cube_y', 'cube_z'], dtype='object')

In [87]:
tra_uids = np.load('for_training/compact/training_uids.npy')
val_uids = np.load('for_training/compact/validation_uids.npy')

In [88]:
tra_rot_uids =  np.intersect1d(tra_uids, rot_tgts.values[:,0])

In [89]:
val_rot_uids =  np.intersect1d(val_uids, rot_tgts.values[:,0])

In [90]:
tra_rot_uids

array([101807., 101808., 101811., ..., 984429., 984431., 984433.])

In [91]:
tra_uids

array([101807., 101808., 101809., ..., 984429., 984431., 984433.])

In [92]:
(1-rot_tgts[['# ids']].iloc[:,0].isin(tra_uids)).sum()

878

In [54]:
imgs = np.zeros([tra_rot_uids.shape[0], 224,224,3], dtype=np.float32)
for i,uid in enumerate(tra_rot_uids.astype(int)):
    try:
        dimg = np.load('../Images/{}_RS_depth.npy'.format(uid)) 
        dimg = dimg*0.00012498664727900177 # now in meters
        dimg[dimg>0.75] = 0 # Set stuff too far away to zero
        dimg = dimg/0.75
        
        dimg = np.roll(dimg, 25, axis=1)
        dimg = np.roll(dimg, -5, axis=0)
        dimg = clipped_zoom(dimg, 1.3)
        
        dimg = dimg[::2, ::2]
        dimg = dimg[-224:, 32:32+224]
        
        cimg = np.load('../Images/{}_RS_color.npy'.format(uid))[::2,::2]
        cimg = cimg[-224:, 32:32+224]
        cimg = cimg.astype(np.float32)

        cimg[:,:,2] = dimg
        imgs[i] = cimg
    except:
        print('Stopping at iteration {} with uid {} because of an exception'.format(i,uid))
        break
max_d = imgs[:,:,:,2].max()
imgs[:,:,:,2] = imgs[:,:,:,2]/max_d * 255

/home/vreyesos/.local/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [61]:
np.save('training_image_rot_data_compact.npy', imgs)

In [62]:
imgs = np.zeros([val_rot_uids.shape[0], 224,224,3], dtype=np.float32)
for i,uid in enumerate(val_rot_uids.astype(int)):
    try:
        dimg = np.load('../Images/{}_RS_depth.npy'.format(uid)) 
        dimg = dimg*0.00012498664727900177 # now in meters
        dimg[dimg>0.75] = 0 # Set stuff too far away to zero
        dimg = dimg/0.75
        
        dimg = np.roll(dimg, 25, axis=1)
        dimg = np.roll(dimg, -5, axis=0)
        dimg = clipped_zoom(dimg, 1.3)
        
        dimg = dimg[::2, ::2]
        dimg = dimg[-224:, 32:32+224]
        
        cimg = np.load('../Images/{}_RS_color.npy'.format(uid))[::2,::2]
        cimg = cimg[-224:, 32:32+224]
        cimg = cimg.astype(np.float32)

        cimg[:,:,2] = dimg
        imgs[i] = cimg
    except:
        print('Stopping at iteration {} with uid {} because of an exception'.format(i,uid))
        break
max_d = imgs[:,:,:,2].max()
imgs[:,:,:,2] = imgs[:,:,:,2]/max_d * 255

In [63]:
np.save('validation_image_rot_data_compact.npy', imgs)

In [73]:
tra_mask = rot_tgts.iloc[:,0].isin(tra_uids)
tra_tgts = rot_tgts[tra_mask][['# ids', 'rot_voxel']].values
np.save('training_rot_targets_compact.npy', tra_tgts)
tra_pos_inp = rot_tgts[tra_mask][['# ids', 'cube_x', 'cube_y', 'cube_z']].values
np.save('training_rot_inputs_compact.npy', tra_pos_inp)

In [96]:
val_mask = rot_tgts.iloc[:,0].isin(val_uids)
val_tgts = rot_tgts[val_mask][['# ids', 'rot_voxel']].values
np.save('validation_rot_targets_compact.npy', val_tgts)
val_pos_inp = rot_tgts[val_mask][['# ids', 'cube_x', 'cube_y', 'cube_z']].values
np.save('validation_rot_inputs_compact.npy', val_pos_inp)

In [81]:
np.argmax(val_rot_uids > 956594)

857

In [82]:
val_rot_uids

array([101847, 102884, 103071, 103075, 103204, 103302, 103333, 108979,
       108992, 116809, 116825, 123454, 123482, 123543, 125902, 126024,
       130074, 132092, 132131, 132181, 132192, 132391, 132453, 134172,
       136570, 136573, 136579, 141549, 141573, 141625, 143075, 143200,
       143356, 147634, 147713, 147718, 147920, 163835, 163861, 164729,
       165322, 165348, 165634, 165849, 166007, 166719, 166758, 168698,
       168702, 168763, 168785, 168809, 173079, 173154, 173281, 173387,
       173438, 173445, 173894, 176424, 177741, 182907, 186210, 186314,
       186390, 187192, 187229, 191865, 191873, 191915, 193349, 193356,
       194542, 199730, 202557, 202561, 202599, 202606, 202700, 202741,
       203235, 203348, 203397, 203437, 203475, 203509, 220205, 220243,
       220304, 223616, 223653, 223661, 223753, 227670, 227671, 227691,
       227724, 228475, 228507, 238443, 238820, 240078, 240124, 240150,
       241781, 244990, 245056, 255934, 260304, 264528, 264561, 267772,
      